In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import sys
from datetime import datetime
sys.path.append('../../')

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow

np.random.seed(0)

# seperate out training and validation datasets

In [ ]:
# https://stackoverflow.com/questions/56257429/randomly-mask-set-nan-x-of-data-points-in-huge-xarray-dataarray

train_dir = Path('../../data/bootstrap/training')
val_dir = Path('../../data/bootstrap/validation')

train_dir.mkdir(parents = True, exist_ok = True)
val_dir.mkdir(parents = True, exist_ok = True)

for fp in Path('../../Lidar_s1_stacks').glob('*.nc'):
    ds = xr.open_dataset(fp)
    mask = xr.zeros_like(ds)['fcf'].rename('mask')
    mask.data = np.random.rand(*mask.data.shape) < 0.2
    mask = mask.broadcast_like(ds['s1'])
    train = ds.where(~mask)
    val = ds.where(mask)

    train.to_netcdf(train_dir.joinpath(fp.name.replace('.nc','.train.nc')))
    val.to_netcdf(val_dir.joinpath(fp.name.replace('.nc','.val.nc')))

# loop through .nc and resample and calculate all possible retrieved snowdepths

In [ ]:
np.random.seed(0)
site_dir = Path('/Users/zachkeskinen/Documents/spicy-snow/scripts/optimize/param_sds/Mores_2021-03-15')
ds = xr.open_dataset(next(site_dir.glob('*')))
idx = ds['lidar-sd'].rename('mask')
idx.data = np.random.rand(*idx.data.shape) < 0.2
idx = idx.broadcast_like(ds)
for fp in site_dir.glob('*'):
    ds = xr.open_dataset(fp)
    a,b,c = fp.stem.split('_')
    train = ds.where(~idx)
    val = ds.where(idx)


In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import box
from pathlib import Path
from datetime import datetime
from scipy.stats import pearsonr
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

import sys
sys.path.append('../..')

from spicy_snow.processing.snow_index import calc_delta_cross_ratio, calc_delta_gamma, \
    clip_delta_gamma_outlier, calc_snow_index, calc_snow_index_to_snow_depth
from spicy_snow.processing.wet_snow import id_newly_wet_snow, id_wet_negative_si, \
    id_newly_frozen_snow, flag_wet_snow

# Create parameter space
A = np.arange(1, 3.1, 0.5)
B = np.arange(0, 1.01, 0.1)
C = np.arange(0, 1.001, 0.01)

files = Path('../../data/bootstrap/training/').glob('*.nc')
for f in files:

    # get dataset
    ds_name = f.name.split('stacks/')[-1].split('.')[0]
    print(datetime.now(), f' -- starting {ds_name}')
    ds_ = xr.open_dataset(f).load()
    dataset = ds_[['s1','deltaVV','ims','fcf','lidar-sd']]

    # find closest timestep to lidar
    td = abs(pd.to_datetime(dataset.time) - pd.to_datetime(dataset.attrs['lidar-flight-time']))
    closest_ts = dataset.time[np.argmin(td)]

    # Brute-force processing loop
    for a in tqdm(A):
        # print(f'A: {a}')
        a = np.round(a, 2)
        ds = calc_delta_cross_ratio(dataset, A = a)
        for b in B:
            b = np.round(b, 2)
            # print(f'    B: {b}')
            ds = calc_delta_gamma(ds, B = b, inplace=False)
            ds = clip_delta_gamma_outlier(ds)
            ds = calc_snow_index(ds)
            ds = id_newly_wet_snow(ds)
            ds = id_wet_negative_si(ds)
            ds = id_newly_frozen_snow(ds)
            ds = flag_wet_snow(ds)
            for c in C:
                c = np.round(c, 2)
                # print(f'        c: {c}')
                # print(f'A={a}; B={b}; C={c}')

                ds = calc_snow_index_to_snow_depth(ds, C = c)

                sub = ds.sel(time = closest_ts)[['snow_depth', 'wet_snow', 'lidar-sd']]
                sub.to_netcdf(f'./param_sds/{ds_name}/{a}_{b}_{c}.nc')